# Text Preprocessing

## Imports

In [1]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestClassifier
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import re
import spacy

nlp = spacy.load('en_core_web_sm')

## Load Data

In [2]:
def load_data(path):
    return pd.read_csv(path, header = 0, delimiter = '\t', quoting = 3)

In [3]:
# Train Data
train_data = load_data(path = 'Data/Raw/labeledTrainData.tsv')
print('Train Data Shape:', train_data.shape)
print('Train Data Valeus:', train_data.columns.values)

# Test Data
test_data = load_data(path = 'Data/Raw/testData.tsv')
print('Test Data Shape:', test_data.shape)
print('Test Data Valeus:', test_data.columns.values)

Train Data Shape: (25000, 3)
Train Data Valeus: ['id' 'sentiment' 'review']
Test Data Shape: (25000, 2)
Test Data Valeus: ['id' 'review']


## Preprocess Data

In [4]:
# Get stopwords list    
stop_words = set(nlp.Defaults.stop_words)


def cleanup(review):
    
    # Remove Markups
    review =  BeautifulSoup(review).get_text()

    # Remove Numbers
    review = re.sub('[^a-zA-Z]', ' ', review)

    # Conver to lowercase
    review = review.lower()

    # Lemmatize
    review = [token.lemma_ for token in nlp(review)]

    # Remove stop words
    review = [word for word in review if not word in stop_words]

    # Rejoin the review words into one string
    review = ' '.join(review)

    return review

def process_data(data, data_type):
    
    # Create a new DataFrame
    cleaned_data = data.copy()    
    
    reviews = []
    for i, review in enumerate(data['review']):
        cleaned_review = cleanup(review)
        reviews.append(cleaned_review)

        if i % 100 == 0:
            print(f'Processing "{data_type}", {i} Review...')
            
    cleaned_data['review'] = reviews
    
    return cleaned_data

In [ ]:
# Train Data
cleaned_train_data = process_data(train_data, data_type = 'Train Data')
cleaned_train_data.head(5)

# Test Data
cleaned_test_data = process_data(data = test_data, data_type = 'Test Data')
cleaned_test_data.head(5)

Processing "Train Data", 0 Review...
Processing "Train Data", 100 Review...
Processing "Train Data", 200 Review...
Processing "Train Data", 300 Review...
Processing "Train Data", 400 Review...
Processing "Train Data", 500 Review...
Processing "Train Data", 600 Review...
Processing "Train Data", 700 Review...
Processing "Train Data", 800 Review...
Processing "Train Data", 900 Review...
Processing "Train Data", 1000 Review...
Processing "Train Data", 1100 Review...


## Save cleaned data

In [ ]:
cleaned_train_data.to_csv('Data/Processed/cleaned_labeledTrainData.csv', index = False)
cleaned_test_data.to_csv('Data/Processed/cleaned_testData.csv', index = False)

## Load the cleaned data

In [ ]:
cleaned_train_data = pd.read_csv('Data/Processed/cleaned_labeledTrainData.csv')
cleaned_test_data = pd.read_csv('Data/Processed/cleaned_testData.csv')

cleaned_train_reviews = cleaned_train_data['review']
cleaned_test_reviews = cleaned_test_data['review']

## Vectorize Data

### Train Data

In [ ]:
vectorizer = CountVectorizer(max_features = 5000)      # Top 5000 frequent words

train_data_features = vectorizer.fit_transform(cleaned_train_reviews)
train_data_features = train_data_features.toarray()

train_data_features.shape

In [ ]:
vectorizer.get_feature_names()

### Test Data

In [ ]:
test_data_features = vectorizer.transform(cleaned_test_reviews)

## Train the Model (Random Forest)

In [ ]:
random_forest = RandomForestClassifier(n_estimators = 100)
random_forest = random_forest.fit(train_data_features, cleaned_train_data['sentiment'])

## Make Prediction

In [ ]:
predictions = random_forest.predict(test_data_features)

## Save Predictions

In [ ]:
predictions_df = pd.DataFrame(data = {'id': cleaned_test_data['id'], 'sentiment': predictions})
predictions_df.to_csv('Data/Processed/Submission.csv', index = False)
predictions_df.head(10)